In [1]:
import sys
import phrasetopicminer as ptm

print("Python:", sys.version)
print("Executable:", sys.executable)
print("PTM version:", ptm.__version__)
print("PTM module file:", ptm.__file__)
print("Available top-level names:", [n for n in dir(ptm) if not n.startswith("_")])


Python: 3.10.10 (main, Oct 24 2024, 22:07:07) [Clang 16.0.0 (clang-1600.0.26.3)]
Executable: /tmp/ptm-test/bin/python
PTM version: 0.1.0
PTM module file: /Users/ahmadhashemi/Dropbox/DS-Projects/PhraseTopicMiner/phrasetopicminer/__init__.py
Available top-level names: ['ClusterLabelingInput', 'ClusterLabelingResult', 'LabeledTopicCluster', 'PackageNotFoundError', 'PhraseMiner', 'PhraseRecord', 'TopicCluster', 'TopicCoreResult', 'TopicLabelModel', 'TopicLabeler', 'TopicLabelingResult', 'TopicModeler', 'TopicTimelineBuilder', 'TopicTimelineResult', 'build_phrase_sentence_examples_from_occurrences', 'make_datamapplot_interactive', 'make_datamapplot_static', 'phrase_miner', 'plot_phrase_bubble_map', 'plot_phrase_treemap', 'plot_topic_timeline', 'topic_labeler', 'topic_modeler', 'topic_timeline', 'topic_viz', 'version', 'visualization_datamap']


In [2]:
from phrasetopicminer import (
    PhraseMiner, 
    TopicModeler, 
    TopicLabeler, 
    make_datamapplot_interactive,
    TopicTimelineBuilder
)

# 1) Phrase mining
miner = PhraseMiner(
    method="spacy",
    spacy_model="en_core_web_lg",
    include_verb_phrases=True,
    clean_markdown=True,
)

In [3]:
text = """
1. Aspects of the mind
Thought is when we absorb what happens around us so that we can deal with it effectively according to our plans and desires. Thinking is using information, like forming concepts, problem solving, reasoning and making choices.

Memory is when we store information in our minds, and can later recall it.

Imagination is the **ability to invent worlds inside the mind**, complete or not. The mind makes these by *drawing on experience in the shared world*.

Consciousness is knowing that we exist and the world exists, and being able to understand what happens around us.

2. Mental health
Just like the body, a mind can be healthy. The measure of this is called mental health. According to the World Health Organization (WHO), there is not one way to measure mental health in all people, because there are many things in our environment that might make what is mentally healthy different from one person to another. In general, most experts agree that "mental health" and "mental illness" are not opposites. In other words, not having a mental illness does not mean you are in good mental health.
"""

text1 = """
One way to study mental health is by looking at how well a person lives. Signs of mental health include: feeling capable and happy, being able to handle normal levels of stress, making and keeping friends, leading an independent life, and being able to recover from difficult situations.
"""

text2 = """
Philosophy
Philosophy of mind is the branch of philosophy that studies the nature of the mind and how it is linked to the body. The main problem is how the mind is related to the body, but there are also questions about the nature of the mind that do not talk about its relation to the physical body.[4]

Dualism and monism are the two main ways people try to solve the mind-body problem. Dualism is when people believe that the mind and body are in some way separate from each other. It can be traced back to Plato,[5] Aristotle,[6][7][8] and the Samkhya and Yoga schools of Hindu philosophy,[9] but it was most precisely formulated by René Descartes in the 17th century.[10]

Monism is the belief that mind and body are not physiologically and ontologically distinct kinds of entities. This view was first seen in Western philosophy by Parmenides in the 5th century BC and was later held by the 17th-century rationalist Baruch Spinoza.[11] According to Spinoza, mind and body are two parts of a larger being.
"""

text3 = """
Idealists think that the mind is all that exists and that the outside world is actually made up by the mind. Physicalists think that everything can be expressed by what is physical. Neutral monists believe that everything can be either mental or physical depending how you see it. For example, a red spot on a wall is physical, because it is an actual thing depending on the physical wall, but it is mental because our brain responds to the colour. The most common monisms in the 20th and 21st centuries have all been different kinds of physicalism, including behaviorism.[1][2][4]

Psychology
Psychology is the study of the way we think, feel and act. It involves the scientific study of processes such as perception, cognition, feelings, personality, as well as things around us that might affect the way we think. From this study, psychologists try to form rules for why we act the way we do. Psychology also includes using this knowledge to help solve problems of everyday life and treat mental health problems.

Social psychology and group behaviour
Social psychology is the study of how we think, feel and act in groups of other people. Most people who study social psychology are either psychologists or sociologists.
"""

texts = [text, text1, text2, text3]


In [6]:
%%time
np_counter, vp_counters, phrase_records, sentences_by_doc = miner.mine_phrases_with_types(texts)

print("Total NP phrases:", len(np_counter))
print("Total VP phrases:", len(vp_counters))
print("Number of phrase records:", len(phrase_records))
print("Docs / sentences:", [len(sents) for sents in sentences_by_doc])

Total NP phrases: 90
Total VP phrases: 4
Number of phrase records: 201
Docs / sentences: [11, 2, 8, 11]
CPU times: user 515 ms, sys: 4.84 ms, total: 520 ms
Wall time: 518 ms


In [7]:
# 2) Topic modeling (core clustering API)
modeler = TopicModeler(
    embedding_backend="sentence_transformers",   # or "spacy", "openai", "cohere", "custom"
    embedding_model="all-MiniLM-L6-v2",
    embedding_fn=None,    # only if embedding_backend="custom"
    spacy_nlp=None,       # only if embedding_backend="spacy"
    random_state=42,
)

core = modeler.fit_core(
    # --- required core inputs ---
    phrase_records=phrase_records,
    sentences_by_doc=sentences_by_doc,

    # --- phrase filtering options ---
    include_kinds={"NP", "VP"},          # only NP; use {"NP", "VP"} to include both
    include_patterns={"BaseNP", "NP+PP", "NP+multiPP", 
                      "VerbObj", "VerbPP", "SubjVerb"
                      },  # or e.g. {"BaseNP", "NP+PP"}
    min_freq_unigram=3,            # threshold for 1-word phrases
    min_freq_bigram=1,             # threshold for 2-word phrases
    min_freq_trigram_plus=1,       # threshold for >=3-word phrases

    # --- geometric pipeline options ---
    pca_n_components=10,           # 0 or None if you want to skip PCA
    cluster_geometry="umap_2d",    # "umap_nd" or "umap_2d"
    umap_n_neighbors=5,
    umap_min_dist=0.1,
    umap_cluster_n_components=10,  # target dim for clustering (if using umap_nd)

    # --- clustering options ---
    clustering_algorithm="hdbscan",   # "hdbscan" or "kmeans"
    hdbscan_min_cluster_size=5,
    hdbscan_min_samples=None,
    hdbscan_metric="euclidean",
    kmeans_max_clusters=15,          # used only if clustering_algorithm="kmeans"

    # --- visualization geometry ---
    viz_reducer="tsne_2d",           # "same", "umap_2d", or "tsne_2d"
    tsne_perplexity=30.0,
    tsne_learning_rate=200.0,
    tsne_n_iter=1000,

    # --- cluster representatives ---
    top_n_representatives=10,

    verbose=True,
)

# 3Build timeline
timeline_builder = TopicTimelineBuilder(
    timeline_mode="reading_time",
    speech_rate_wpm=200,
    log_fn=print,  # or st.markdown, rich console logger, etc.
)

timeline = timeline_builder.build(core_result=core,
                                  sentences_by_doc=sentences_by_doc)


[TopicModeler] Step 1/12 – filtering PhraseRecords by kind/pattern...
[TopicModeler]   → kept 182 occurrences after filtering.
[TopicModeler] Step 2/12 – built occurrence map for 154 phrases (freq source: reconstructed_from_records).
[TopicModeler] Step 3/12 – phrase DataFrame has 154 rows before n-gram filtering.
[TopicModeler] Step 4/12 – 107 phrases remain after n-gram thresholds.
[TopicModeler] Step 5/12 – phrase_sentences built for 107 phrases.
[TopicModeler] Step 6/12 – embedding 107 phrases...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

[TopicModeler]   → embeddings computed with backend='sentence_transformers'.
[TopicModeler] Step 7/12 – PCA denoising to 10 dims.


/tmp/ptm-test/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[TopicModeler] Step 8/12 – UMAP geometry (umap_2d).


/tmp/ptm-test/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/tmp/ptm-test/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[TopicModeler] Step 9/12 – clustering with 'hdbscan'.
[TopicModeler] Step 10/12 – computing 2D viz embedding via 'tsne_2d'.
[TopicModeler] Step 11/12 – aggregating TopicCluster objects.
[TopicModeler] Step 12/12 – assembling TopicCoreResult.
▶ Building sentence-level timeline…
▶ Linking phrases to sentences…
▶ Aggregating sentences per cluster…
✅ TopicTimelineBuilder finished.


In [ ]:
from phrasetopicminer import (
    plot_topic_timeline,
    plot_phrase_bubble_map,
    plot_phrase_treemap,
    make_datamapplot_dataframe,
)

In [ ]:
plot_phrase_bubble_map(core, max_phrases=80, show_text=False)

In [12]:
import phrasetopicminer as ptm

docs = [
    "Topic modeling clusters noun phrases that capture the core ideas in documents.",
    "Phrase-based topic modeling focuses on multi-word expressions rather than single tokens.",
    "The resulting phrase clusters can be visualized as a topic map in 2D space.",
]

miner = ptm.PhraseMiner()
np_counter, vp_counters, phrase_records, sentences_by_doc = miner.mine_phrases_with_types(docs)

modeler = ptm.TopicModeler(
    embedding_backend="sentence_transformers",
    embedding_model="all-MiniLM-L6-v2",
    logger=print,
)

core = modeler.fit_core(
    phrase_records=phrase_records,
    sentences_by_doc=sentences_by_doc,
    pca_n_components=2,   # for this tiny test
    verbose=True,
)

core.phrases_df.head()


[TopicModeler] Step 1/12 – filtering PhraseRecords by kind/pattern...
[TopicModeler]   → kept 8 occurrences after filtering.
[TopicModeler] Step 2/12 – built occurrence map for 8 phrases (freq source: reconstructed_from_records).
[TopicModeler] Step 3/12 – phrase DataFrame has 8 rows before n-gram filtering.
[TopicModeler] Step 4/12 – 4 phrases remain after n-gram thresholds.
[TopicModeler] Step 5/12 – phrase_sentences built for 4 phrases.
[TopicModeler] Step 6/12 – embedding 4 phrases...
[TopicModeler]   → embeddings computed with backend='sentence_transformers'.
[TopicModeler] Step 7/12 – PCA denoising to 2 dims.


/tmp/ptm-test/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/tmp/ptm-test/lib/python3.10/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/tmp/ptm-test/lib/python3.10/site-packages/umap/spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

In [8]:
from phrasetopicminer import smoke_test

result = smoke_test.run_smoke_test(verbose=True)

core = result["core"]
timeline = result["timeline"]

[smoke_test] Starting PhraseTopicMiner smoke test...
[smoke_test] Using 3 small demo documents.
[smoke_test] Creating PhraseMiner and mining phrases...
[smoke_test] Mined 6 phrase records across 3 document(s).
[smoke_test] Total NP: Counter({'structure in collections of documents': 1, 'Phrase': 1, 'topic modeling clusters noun': 1, 'coherent themes': 1, 'phrase clusters as a topic map in 2D space': 1}), total VP: {}
[smoke_test] Creating TopicModeler and fitting core model...
[TopicModeler] Step 1/12 – filtering PhraseRecords by kind/pattern...
[TopicModeler]   → kept 6 occurrences after filtering.
[TopicModeler] Step 2/12 – built occurrence map for 6 phrases (freq source: reconstructed_from_records).
[TopicModeler] Step 3/12 – phrase DataFrame has 6 rows before n-gram filtering.
[TopicModeler] Step 4/12 – 6 phrases remain after n-gram thresholds.
[TopicModeler] Step 5/12 – phrase_sentences built for 6 phrases.
[TopicModeler] Step 6/12 – embedding 6 phrases...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[TopicModeler]   → embeddings computed with backend='sentence_transformers'.


ValueError: n_components=10 must be between 0 and min(n_samples, n_features)=6 with svd_solver='full'